# Note

* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame


* Load the csv exported in Part I to a DataFrame


In [3]:
weather_file = "Output_data/cities.csv"
weather_data = pd.read_csv(weather_file)

weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tuktoyaktuk,69.4541,-133.0374,51.80,100,100,5.75,CA,1660666765
1,Pevek,69.7008,170.3133,38.70,67,42,8.81,RU,1660666766
2,Sorong,-0.8833,131.2500,76.23,88,100,3.06,ID,1660666766
3,Khatanga,71.9667,102.5000,41.97,87,1,10.02,RU,1660666766
4,Hasaki,35.7333,140.8333,82.80,80,100,13.27,JP,1660666768
...,...,...,...,...,...,...,...,...,...
533,Ulaanbaatar,47.9077,106.8832,62.73,72,40,8.95,MN,1660666800
534,Jais,26.2500,81.5333,84.70,67,72,5.64,IN,1660666983
535,Pouembout,-21.1333,164.9000,69.37,94,94,4.25,NC,1660666984
536,Hadžići,43.8222,18.2067,76.33,57,20,4.61,BA,1660666985


# Humidity Heatmap


* Configure gmaps.

* Use the Lat and Lng as locations and Humidity as the weight.

* Add Heatmap layer to map.


In [4]:
#Configuring gmaps

gmaps.configure(api_key = g_key)


locations = weather_data[['Lat', 'Lng']]
humidity = weather_data['Humidity']

In [13]:
#Creating the Heatmap

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating = False, max_intensity = 100, point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria



* Narrow down the cities to fit weather conditions.

* Drop any rows will null values.
 

In [17]:
#perfect temp is at least 65 no higer than 79
ideal_weather = weather_data.loc[(weather_data["Max Temp"]<=80) & (weather_data["Max Temp"]>=70)]

ideal_weather = ideal_weather.loc[ideal_weather["Cloudiness"]<=0]

ideal_weather = ideal_weather.loc[(ideal_weather["Humidity"]<=65) & (ideal_weather["Humidity"]>=55)]

ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] <=10]

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,Mehamn,71.0357,27.8492,71.55,64,0,6.91,NO,1660666655
113,Chippewa Falls,44.9369,-91.3929,76.96,59,0,6.91,US,1660666562
234,Casper,42.8666,-106.3131,73.72,65,0,1.99,US,1660666858
497,Jackson,42.4165,-122.8345,74.05,57,0,1.01,US,1660666964


# Hotel Map


* Store into variable named hotel_df.

* Add a "Hotel Name" column to the DataFrame.

* Set parameters to search for hotels with 5000 meters.

* Hit the Google Places API for each city's coordinates.

* Store the first Hotel result into the DataFrame.

* Plot markers on top of the heatmap.


In [18]:
#creating hotel df
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ''

In [19]:
#create a base url and add some parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius": 5000, "types": "lodging", "key": g_key}

In [21]:
#storing hotel

for index,row in hotel_df.iterrows():
    
    lat=row["Lat"]
    lon=row["Lng"]
    
    params["location"] = f"{lat},{lon}"
        
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'Hotel Name']= results[0]['name']
    
    except:
        print ("Missing Data....Skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan
                

In [23]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,Mehamn,71.0357,27.8492,71.55,64,0,6.91,NO,1660666655,Mehamn Arctic Hotel AS
113,Chippewa Falls,44.9369,-91.3929,76.96,59,0,6.91,US,1660666562,AmericInn by Wyndham Chippewa Falls
234,Casper,42.8666,-106.3131,73.72,65,0,1.99,US,1660666858,Ramkota Hotel
497,Jackson,42.4165,-122.8345,74.05,57,0,1.01,US,1660666964,Agate Village Apartments


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map

markers= gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))